# Pairs Trading example

In this Jupyter Notebook, we present a proof-of-concept demonstrating the capabilities of the library. 

Specifically, we implement a simple pair trading algorithm that fetches data and trades on the Bybit testnet. The algorithm identifies pairs of cryptocurrencies with the highest cointegration coefficient, a widely used metric in pairs trading. These pairs are expected to exhibit a long-term price relationship. The strategy exploits short-term price deviations by shorting the overpriced asset and longing the underpriced one whenever their price difference diverges significantly from the spread's moving average.

In [1]:
from read_api_keys import read_api_keys
import itertools
import numpy as np
import pandas as pd
import time
import sys
sys.path.append('../')
from pybit_ms.bybit_client import BybitAPI

We first import the api keys and we initialize the bybit client.

In [2]:
keys = read_api_keys("TAK.txt")
public_key = keys.get('PUBLIC_KEY')
private_key = keys.get('PRIVATE_KEY')

client = BybitAPI(api_key=public_key, api_secret=private_key, testnet=1)
client

BybitAPI(testnet=1)

We then create the coin list in which we will search for cointegrated pairs of cryptocurrencies. We fetch the prices up to 30 days ago, and we also preprocess them normalizing them.

In [3]:
COINS = ["BTC", "ETH", "BNB", "SOL", "XRP", "DOGE", "ADA", "LINK", "DOT", "TRX"]
COIN_PAIRS = list(itertools.combinations(COINS, 2))

data = pd.DataFrame()

for coin in COINS:
    # Get Kline of previous 30 days with interval of 1 hour
    data[f'{coin}'] = client.market.get_kline(category="linear", coin1=coin, coin2="USDT", interval="60", price_type="close", limit=720)
    
    # Convert prices to float
    data[f'{coin}'] = pd.to_numeric(data[f'{coin}'], errors='coerce')
    
    # Normalize the prices
    min_price = data[f'{coin}'].min()
    max_price = data[f'{coin}'].max()
    data[f'{coin}'] = (data[f'{coin}']) - min_price / (max_price - min_price)
    
data

,BTC,ETH,BNB,SOL,XRP,DOGE,ADA,LINK,DOT,TRX
0,83683.899128,3280.128416,694.726044,187.086583,0.065625,-1.144987,-0.828762,19.048641,4.7503,-1.434481
1,82999.999128,3210.268416,667.226044,184.056583,0.160125,-1.138317,-0.725662,18.693641,4.8483,-1.429671
2,83604.699128,3276.608416,663.726044,174.906583,0.047025,-1.134987,-0.786562,17.053641,4.7203,-1.419031
3,85499.999128,3346.468416,676.726044,191.626583,0.218225,-1.148977,-0.697762,18.687641,4.6103,-1.418511
4,30475.499128,3172.868416,676.726044,187.216583,0.187625,-1.155707,-0.823162,17.098641,5.1553,-1.425111
...,...,...,...,...,...,...,...,...,...,...
715,84150.299128,3549.798416,684.226044,220.406583,0.124225,-1.059527,-0.696762,21.409641,6.1923,-1.414901
716,96167.199128,3488.888416,647.726044,224.696583,0.161225,-1.085137,-0.720662,20.180641,5.8333,-1.417901
717,74319.999128,3639.148416,654.726044,215.946583,0.185925,-1.068317,-0.723662,18.610641,6.6783,-1.377211
718,74319.999128,3692.658416,662.226044,207.226583,0.324825,-1.065587,-0.611762,20.556641,6.4113,-1.426621


In [ ]:
!pip install statsmodels

We create functions to find cointegration coefficient and best cointegrated pairs.

We then select the top pair that we will use in our trading strategy

In [4]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

# Calculate cointegration coefficient
def calc_adf(data, coin1, coin2):
    Y = data[f'{coin1}']
    X = data[f'{coin2}']
    X = sm.add_constant(X)  
    model = sm.OLS(Y, X).fit()
    residuals = model.resid
    result = adfuller(residuals)[0]
    return result

# Find pairs with highest cointegration
def coint_pairs(data):  
    adfs = {}
    for pair in COIN_PAIRS:
        coin1 = pair[0]
        coin2 = pair[1]
        adfs[f"{coin1}-{coin2}"] = calc_adf(data, coin1, coin2)
    adfs = dict(sorted(adfs.items(), key=lambda item: item[1], reverse=True))
    return list(adfs.items())

top_pair = coint_pairs(data)[:1][0]
print(top_pair)
coin1 = top_pair[0][:top_pair[0].index('-')]
coin2 = top_pair[0][top_pair[0].index('-')+1:]


('DOGE-LINK', np.float64(-0.9904469554991913))


We can then check here if we have enough assets for our strategy

In [5]:
client.account.get_wallet_balance(accountType="UNIFIED")

Total equity: $102,275.99
BTC: Wallet Balance = 1.000000, USD Value = $92205.37
USDT: Wallet Balance = 9978.946318, USD Value = $10070.62


And we also set our futures leverage for our pairs to 1X (Bybit futures set it automatically to 10X before first usage)


In [12]:
client.trade.set_leverage(category="linear", symbol=f"{coin1}USDT", buy_leverage="1", sell_leverage="1")
client.trade.set_leverage(category="linear", symbol=f"{coin2}USDT", buy_leverage="1", sell_leverage="1")

buy_leverage: 1
sell_leverage: 1
buy_leverage: 1
sell_leverage: 1


## Trading strategy

Here we finally begin our simple algorithm

In [ ]:
# q = 1000	# quantity in USDT we will trade in each order
# open_position=False		# Indicator wheter we have an open position or not

# while True:
# 	# We first retrieve past five days prices
# 	coin1_past_5days = client.market.get_kline(category="linear", coin1=coin1, coin2="USDT", interval="60", price_type="close", limit=120)
# 	coin2_past_5days = client.market.get_kline(category="linear", coin1=coin2, coin2="USDT", interval="60", price_type="close", limit=120)
	
# 	# We convert prices to float
# 	coin1_past_5days = np.array(coin1_past_5days, dtype=float)
# 	coin2_past_5days = np.array(coin2_past_5days, dtype=float)
	
# 	# We normalize prices
# 	min_price1 = coin1_past_5days.min()
# 	max_price1 = coin1_past_5days.max()
# 	coin1_past_5days = (coin1_past_5days - min_price1) / (max_price1 - min_price1)
	
# 	min_price2 = coin2_past_5days.min()
# 	max_price2 = coin2_past_5days.max()
# 	coin2_past_5days = (coin2_past_5days - min_price2) / (max_price2 - min_price2)
	
# 	# We calculate statistics we will use to trigger our stop orders
# 	spread = coin2_past_5days - coin1_past_5days
# 	spread_mean = spread.mean()
# 	spread_std = spread.std()
	
# 	# We calculate bollinger bands to trigger our orders
# 	up_band = spread_mean + 2 * spread_std
# 	low_band = spread_mean - 2 * spread_std

# 	print("Calculated stats...")


# 	# ----------------------------------------------------------------------------------
# 	# -------------------------------------Strategy-------------------------------------
# 	# ----------------------------------------------------------------------------------

# 	if spread[-1] < low_band and not open_position:
# 		# Open short position on coin1
# 		client.trade.place_futures_order(symbol=f"{coin1}USDT", side="Sell", order_type="Market", qty=q, market_unit="quoteCoin")
# 		# Open long position on coin2
# 		client.trade.place_futures_order(symbol=f"{coin2}USDT", side="Buy", order_type="Market", qty=q, market_unit="quoteCoin")
# 		open_position = True
		
# 	if spread[-1] > up_band and not open_position:
# 		# Open long position on coin1
# 		client.trade.place_futures_order(symbol=f"{coin1}USDT", side="Buy", order_type="Market", qty=q, market_unit="quoteCoin")
# 		# Open short position on coin2
# 		client.trade.place_futures_order(symbol=f"{coin2}USDT", side="Sell", order_type="Market", qty=q, market_unit="quoteCoin")
# 		open_position = True

# 	if spread[-2] > spread_mean and spread[-1] < spread_mean and open_position:
# 		# Close both positions
# 		client.trade.close_order(symbol=f"{coin1}USDT", side="Sell", order_type="Market", qty=q, market_unit="quoteCoin")
# 		client.trade.close_order(symbol=f"{coin2}USDT", side="Buy", order_type="Market", qty=q, market_unit="quoteCoin")
# 		open_position = False
	
# 	if spread[-2] < spread_mean and spread[-1] > spread_mean  and open_position:
# 		# Close both positions
# 		client.trade.close_order(symbol=f"{coin1}USDT", side="Buy", order_type="Market", qty=q, market_unit="quoteCoin")
# 		client.trade.close_order(symbol=f"{coin2}USDT", side="Sell", order_type="Market", qty=q, market_unit="quoteCoin")
# 		open_position = False

# 	# display open positions
# 	client.trade.get_positions(category="linear", settle_coin="USDT")
# 	print("Waiting next hour")
# 	time.sleep(3600)
q = 1000
print("Calculated stats...")
# client.trade.place_futures_order(symbol=f"{coin1}USDT", side="Sell", order_type="Market", qty=q, market_unit="quoteCoin")
# client.trade.place_futures_order(symbol=f"{coin2}USDT", side="Buy", order_type="Market", qty=q, market_unit="quoteCoin")
client.trade.get_positions(category="linear", settle_coin="USDT")
print("Waiting next hour")

Calculated stats...


InvalidRequestError: ab not enough for new order (ErrCode: 110007) (ErrTime: 21:44:26).
Request → POST https://api-testnet.bybit.com/v5/order/create: {"category": "linear", "symbol": "LINKUSDT", "side": "Buy", "orderType": "Market", "qty": "1000", "price": null, "timeInForce": "IOC", "marketUnit": "quoteCoin", "positionIdx": null, "orderLinkId": null, "takeProfit": null, "stopLoss": null, "tpTriggerBy": null, "slTriggerBy": null, "tpLimitPrice": null, "slLimitPrice": null, "tpOrderType": null, "slOrderType": null, "tpslMode": null}.

In [5]:
client.trade.cancel_all_orders(category="linear", settle_coin="USDT")

[]

In [6]:
client.trade.get_closed_pnl(category="linear")

,symbol,orderType,side,leverage,orderPrice,avgEntryPrice,avgExitPrice,qty,closedSize,closedPnl,fillCount,orderId,createdTime
0,BTCUSDT,Trade (Market),Buy,10,84538.1,83075.38,82528.1,0.05,0.05,7.17628741,1,id: 69b9ef3a-a644-4cba-b02e-c29cdeb665b0,2025-01-09 00:02:55
